In [1]:
## US Department of Transportation flight data
import pandas as pd

In [2]:
# Download the data file first:
# https://github.com/multidis/hult-sf-bus-challenge/raw/refs/heads/main/data_collection/bts/T_T100_MARKET_ALL_CARRIER.csv
# save it locally. The data covers 2024-01 through 2024-07.
# For a wider time range:
# https://www.transtats.bts.gov/DL_SelectFields.aspx
fcsv = "T_T100_MARKET_ALL_CARRIER.csv"
df = pd.read_csv(fcsv).rename(lambda x: x.lower(), axis=1)

In [3]:
df

,passengers,freight,mail,distance,unique_carrier,unique_carrier_name,origin_airport_id,origin,origin_city_name,origin_state_abr,...,dest_city_name,dest_state_abr,dest_state_nm,dest_country,dest_country_name,year,quarter,month,distance_group,class
0,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15167,TEB,"Teterboro, NJ",NJ,...,"Teterboro, NJ",NJ,New Jersey,US,United States,2024,3,7,1,L
1,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15447,UGN,"Waukegan, IL",IL,...,"Waukegan, IL",IL,Illinois,US,United States,2024,2,4,1,L
2,0.0,0.0,0.0,0.0,2EQ,"J&M Alaska Air Tours, Inc. d/b/a Alaska Air Tr...",13467,MRI,"Anchorage, AK",AK,...,"Anchorage, AK",AK,Alaska,US,United States,2024,2,5,1,F
3,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,11697,FLL,"Fort Lauderdale, FL",FL,...,"Fort Lauderdale, FL",FL,Florida,US,United States,2024,2,6,1,L
4,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,16541,OBE,"Okeechobee, FL",FL,...,"Okeechobee, FL",FL,Florida,US,United States,2024,2,6,1,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184075,87916.0,264772.0,528.0,404.0,DL,Delta Air Lines Inc.,10397,ATL,"Atlanta, GA",GA,...,"Orlando, FL",FL,Florida,US,United States,2024,1,3,1,F
184076,89210.0,111759.0,3023.0,404.0,DL,Delta Air Lines Inc.,10397,ATL,"Atlanta, GA",GA,...,"Orlando, FL",FL,Florida,US,United States,2024,2,5,1,F
184077,90767.0,400244.0,5570.0,1448.0,AS,Alaska Airlines Inc.,14747,SEA,"Seattle, WA",WA,...,"Anchorage, AK",AK,Alaska,US,United States,2024,2,6,3,F
184078,98270.0,418375.0,23601.0,1448.0,AS,Alaska Airlines Inc.,14747,SEA,"Seattle, WA",WA,...,"Anchorage, AK",AK,Alaska,US,United States,2024,3,7,3,F


In [4]:
# top carries flying the largest number of passengers out of the U.S.
top_by_passengers = df\
    .query('origin_country_name == "United States" and dest_country_name != "United States"')\
    .groupby('unique_carrier_name').passengers.sum().nlargest(10)
top_by_passengers

unique_carrier_name
American Airlines Inc.                                         10728346.0
United Air Lines Inc.                                          10181688.0
Delta Air Lines Inc.                                            7988790.0
JetBlue Airways                                                 3277862.0
British Airways Plc                                             2217848.0
Air Canada                                                      2009363.0
Westjet                                                         1665980.0
Concesionaria Vuela Compania De Aviacion SA de CV (Volaris)     1623268.0
Southwest Airlines Co.                                          1489668.0
Lufthansa German Airlines                                       1457575.0
Name: passengers, dtype: float64

In [5]:
# most popular destination country for each carrier
df_dest = df.query('origin_country_name == "United States" and dest_country_name != "United States"')\
    .groupby(['unique_carrier_name', 'dest_country_name']).passengers.sum()\
    .groupby(level=0, group_keys=False).nlargest(1)
df_dest

unique_carrier_name                  dest_country_name
1263343 Alberta Inc. d/b/a Lynx Air  Canada                 27253.0
1903 Aviation                        Portugal                   5.0
ABS JETS A.S                         Czech Republic             4.0
ABSA-Aerolinhas Brasileiras          Argentina                  0.0
ABX Air Inc                          Belgium                    0.0
                                                            ...    
Western Global                       Bahrain                    0.0
Westjet                              Canada               1665980.0
World2Fly Portugal SA                Spain                    263.0
Xiamen Airlines Co., Ltd.            China                  34272.0
ZIPAIR Tokyo Inc.                    Japan                 158149.0
Name: passengers, Length: 241, dtype: float64

In [6]:
# most popular destinations for each top carrier specifically
df_dest[top_by_passengers.index]

unique_carrier_name                                          dest_country_name 
American Airlines Inc.                                       Mexico                2420906.0
United Air Lines Inc.                                        Mexico                1815916.0
Delta Air Lines Inc.                                         Mexico                1365350.0
JetBlue Airways                                              Dominican Republic    1076404.0
British Airways Plc                                          United Kingdom        2217638.0
Air Canada                                                   Canada                2009363.0
Westjet                                                      Canada                1665980.0
Concesionaria Vuela Compania De Aviacion SA de CV (Volaris)  Mexico                1623268.0
Southwest Airlines Co.                                       Mexico                 842432.0
Lufthansa German Airlines                                    Germany               

In [7]:
## the queries above were adopted from the following tutorial:
## https://github.com/stefmolin/pandas-workshop/blob/main/asynchronous_lab/solutions.ipynb
## Find more useful examples in there by Stefanie Molin, https://github.com/stefmolin

In [8]:
## if you prefer the SQL-way:
## conda install -c conda-forge pandasql
from pandasql import sqldf

In [9]:
query = """
  SELECT * 
  FROM df
  WHERE origin_country_name == "United States"
  LIMIT 5;
"""

In [10]:
dfsel = sqldf(query, locals())
dfsel

,passengers,freight,mail,distance,unique_carrier,unique_carrier_name,origin_airport_id,origin,origin_city_name,origin_state_abr,...,dest_city_name,dest_state_abr,dest_state_nm,dest_country,dest_country_name,year,quarter,month,distance_group,class
0,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15167,TEB,"Teterboro, NJ",NJ,...,"Teterboro, NJ",NJ,New Jersey,US,United States,2024,3,7,1,L
1,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15447,UGN,"Waukegan, IL",IL,...,"Waukegan, IL",IL,Illinois,US,United States,2024,2,4,1,L
2,0.0,0.0,0.0,0.0,2EQ,"J&M Alaska Air Tours, Inc. d/b/a Alaska Air Tr...",13467,MRI,"Anchorage, AK",AK,...,"Anchorage, AK",AK,Alaska,US,United States,2024,2,5,1,F
3,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,11697,FLL,"Fort Lauderdale, FL",FL,...,"Fort Lauderdale, FL",FL,Florida,US,United States,2024,2,6,1,L
4,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,16541,OBE,"Okeechobee, FL",FL,...,"Okeechobee, FL",FL,Florida,US,United States,2024,2,6,1,L


In [11]:
## if you prefer the SQL way to begin with: example tutirial with the same type of data
## https://github.com/dannguyen/bts-transstats-t100-domestic-demo